In [1]:
cd ..

C:\Users\Agustin\Dropbox\Doctorado\NeuroData\PyLeech


In [2]:
cd ..

C:\Users\Agustin\Dropbox\Doctorado\NeuroData


In [3]:
%matplotlib qt5

In [4]:
import PyLeech.Utils.CrawlingDatabaseUtils as CDU
from PyLeech.Utils.burstStorerLoader import BurstStorerLoader
import PyLeech.Utils.burstUtils as burstUtils
import numpy as np
import matplotlib.pyplot as plt
import PyLeech.Utils.NLDUtils as NLD
import scipy.signal as spsig
import os.path
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import FastICA, FactorAnalysis, PCA
from sklearn.preprocessing import StandardScaler
import PyLeech.Utils.AbfExtension as abfe
import PyLeech.Utils.spikeUtils as spikeUtils
import more_itertools as mit

In [5]:
cdd = CDU.loadDataDict()
file_list = list(cdd)
trace_list = []
emb_list = []
ran_files = []
run_dict = {}
a = [print(i, fn) for fn, i in zip(file_list, range(len(file_list)))]

0 RegistrosDP_PP\18n05010.pklspikes
1 RegistrosDP_PP\2018_10_11_0002.pklspikes
2 RegistrosDP_PP\2018_12_03_0005.pklspikes
3 RegistrosDP_PP\2018_12_04_0003_0004_0005_0006b.pklspikes
4 RegistrosDP_PP\2018_12_06_0005.pklspikes
5 RegistrosDP_PP\2018_12_13_0015.pklspikes
6 RegistrosDP_PP\NS_DP_PP_0.pklspikes
7 RegistrosDP_PP\NS_T_DP_PP_0_cut.pklspikes
8 RegistrosDP_PP\NS_T_DP_PP_1.pklspikes
9 RegistrosDP_PP\2014_09_25_0008.pklspikes
10 RegistrosDP_PP\2019_01_28_0001.pklspikes
11 RegistrosDP_PP\2018_11_06_0004.pklspikes
12 RegistrosDP_PP\2018_12_03_0000_0001.pklspikes
13 RegistrosDP_PP\14217000.pklspikes
14 RegistrosDP_PP\2017_12_08_0002.pklspikes
15 RegistrosDP_PP\cont10.pklspikes
16 RegistrosDP_PP\2018_12_13_0001.pklspikes
17 RegistrosDP_PP\2019_07_22_0002.pklspikes
18 RegistrosDP_PP\2019_07_22_0009.pklspikes
19 RegistrosDP_PP\2019_07_22_0011.pklspikes
20 RegistrosDP_PP\2019_07_23_0004.pklspikes
21 RegistrosDP_PP\2019_07_23_0008.pklspikes
22 RegistrosDP_PP\2019_07_23_0014.pklspikes
23 Regi

In [6]:
del cdd["RegistrosDP_PP\\NS_DP_PP_0.pklspikes"]
del cdd["RegistrosDP_PP\\NS_T_DP_PP_0_cut.pklspikes"]
del cdd["RegistrosDP_PP\\NS_T_DP_PP_1.pklspikes"]
del cdd["RegistrosDP_PP\\2019_01_28_0001.pklspikes"]

In [7]:
binning_dt = 1
spike_kernel_sigma = 1.5
n_components = 3

sc = StandardScaler()
ica = FastICA(n_components=n_components, max_iter=1000)
fa = FactorAnalysis(n_components=n_components)
pca = PCA(n_components=n_components)
ica_dict = {}
fa_dict = {}
pca_dict = {}

In [13]:
for fn, data in cdd.items():
    if ('2019_08_28_0005' not in fn) and ('2019_08_30_0003' not in fn):
        continue
#     try:
#         ns_channel = [key for key, items in data['channels'].items() if 'NS'==items][0] 
#         print("Running %s" % fn)
#     except IndexError:
#         continue
    
    arr_dict, time_vector1, fs = abfe.getArraysFromAbfFiles(fn, ['IN5'])
        
        
#     NS_kernel = NLD.generateGaussianKernel(sigma=spike_kernel_sigma, time_range=20, dt_step=1/fs)
#     conv_NS = spsig.fftconvolve(arr_dict[ns_channel], NS_kernel, mode='same')[::int(binning_dt * fs)]
    time_vector1 = time_vector1[::int(binning_dt * fs)]
    del arr_dict

    burst_object = BurstStorerLoader(fn, 'RegistrosDP_PP', 'load')

    good_neurons = [neuron for neuron, neuron_dict in data['neurons'].items() if neuron_dict['neuron_is_good']]

    spike_freq_array = burstUtils.processSpikeFreqDict(burst_object.spike_freq_dict, step=int(binning_dt * fs) / fs,
                                                       num=time_vector1.shape[0], selected_neurons=good_neurons,
                                                       time_length=burst_object.time[-1])

    smoothed_sfd = {}
    burst_array = []

    kernel = NLD.generateGaussianKernel(sigma=spike_kernel_sigma, time_range=20, dt_step=binning_dt)
#     burst_array.append(conv_NS)
    for key, items in spike_freq_array.items():
        smoothed_sfd[key] = np.array([items[0], spsig.fftconvolve(items[1], kernel, mode='same')])
        burst_array.append(smoothed_sfd[key][1])
    burst_array = np.array(burst_array).T


    fig, axl = burstUtils.plotFreq(burst_object.spike_freq_dict, draw_list=good_neurons, scatter_plot=True,
#                         optional_trace=[time_vector1, conv_NS], 
                                   outlier_thres=3.5)
    fig.suptitle(fn)
    for ax in axl:
        ax.axvline(cdd[fn]["crawling_intervals"][0][0], c='r')
        ax.axvline(cdd[fn]["crawling_intervals"][-1][-1], c='r')

    spike_idxs = NLD.getSpikeIdxs(smoothed_sfd, cdd[fn]["crawling_intervals"])
    scaled_data = sc.fit_transform(burst_array[spike_idxs])
    ica_dict[fn] = ica.fit_transform(scaled_data)
    fa_dict[fn] = fa.fit_transform(scaled_data)
    pca_dict[fn] = pca.fit_transform(scaled_data)

In [14]:
model_dict = {'FA': fa_dict, 'PCA': pca_dict, "ICA": ica_dict}

In [17]:
fig0, ax0 = plt.subplots(3, 4)
# fig1, ax1 = plt.subplots(len(current_dict), 1)

for model, current_dict in model_dict.items():
    j = 0
    max_len = max([items.shape[0] for key, items in current_dict.items()])
    for fn, items in current_dict.items():
        
        # fig = plt.figure()
        # ax = Axes3D(fig)

        print(items.shape)
    #     fig, ax = plt.subplots(2, 1)
        # num = 3
        # idxs = np.linspace(0, items.shape[0], num+1, dtype=int)
        # colors = ['r', 'k', 'g', 'b', 'c']
        # for i in range(idxs.shape[0]-1):
        #     if i==1: continue


            # ax.scatter3D(items[idxs[i]:idxs[i+1],0], items[idxs[i]:idxs[i+1],1], items[idxs[i]:idxs[i+1],2], color=colors[i], s=5)
            # ax.scatter(items[idxs[i]:idxs[i+1],0], items[idxs[i]:idxs[i+1],1], color=colors[i], s=5)

        ax0[j].plot(items[:, 0], color='k')
        ax0[j].plot(items[:, 1], color='r')
        ax0[j].set_xlim([0, max_len])

    #     ax1[j].set_xlim([0, max_len])

        j += 1

(725, 3)


AttributeError: 'numpy.ndarray' object has no attribute 'plot'

In [86]:
### 3d representation

num_colors = 6
for model, current_dict in model_dict.items():
    if model == 'PCA':
        
        i = 1
        for key, items in current_dict.items():
            fig = plt.figure() 
            ax = Axes3D(fig)
#             ax = fig.add_subplot(3, 4, i, projection='3d')
    #         ax[i, j].scatter(items[:, 0], items[:, 1], c=plt.cm.jet(np.linspace(0, 1, items.shape[0])), s=15)
            ax.scatter(items[:, 0], items[:, 1], items[:, 2], c=plt.cm.jet(np.repeat(np.linspace(.1, .9, num_colors), np.ceil(items.shape[0]/num_colors))[:items.shape[0]]), s=15)
            ax.set_title(os.path.basename(os.path.splitext(key)[0]))
            ax.xaxis.set_visible(False)
            ax.yaxis.set_visible(False)

            i += 1
#         fig.suptitle(model)
#         fig.subplots_adjust(right=0.8)
#         cbar_ax = fig.add_axes([0.9, 0.15, 0.025, 0.7])
#         fig.colorbar(sm, ticks=np.linspace(0, 1, N), boundaries=np.arange(-0.05, 1.1, .05), cax=cbar_ax)

In [20]:
### 2d representation


N = 21
cmap = plt.cm.brg
norm = matplotlib.colors.Normalize(vmin=0, vmax=1)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

num_colors = 10
for model, current_dict in model_dict.items():
    if model == 'PCA':
#         fig, ax = plt.subplots(3, 4)
        j = 0 
        i = 0
        for key, items in current_dict.items():
            fig, ax = plt.subplots()
    #         ax[i, j].scatter(items[:, 0], items[:, 1], c=plt.cm.jet(np.linspace(0, 1, items.shape[0])), s=15)
#             ax[i, j].scatter(items[:, 0], items[:, 1], c=cmap(np.repeat(np.linspace(0, 1, num_colors), np.ceil(items.shape[0]/num_colors))[:items.shape[0]]), s=8)
#             ax[i, j].set_title(os.path.basename(os.path.splitext(key)[0]))
            ax.scatter(items[:, 0], items[:, 1], c=cmap(np.repeat(np.linspace(0, 1, num_colors), np.ceil(items.shape[0]/num_colors))[:items.shape[0]]), s=8)
            ax.set_title(os.path.basename(os.path.splitext(key)[0]))
            j += 1

            if j==4:
                j = 0
                i += 1
        fig.suptitle(model)
        fig.subplots_adjust(right=0.8)
        cbar_ax = fig.add_axes([0.9, 0.15, 0.025, 0.7])
        fig.colorbar(sm, ticks=np.linspace(0, 1, N), boundaries=np.arange(-0.05, 1.1, .05), cax=cbar_ax)


In [46]:
import seaborn as sns

sequential_colors = [
            'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
            'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
            'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']

num_colors = 6

for key, items in pca_dict.items():
    fig, ax = plt.subplots()
    i = 0
    for elem in np.array_split(items, num_colors):
        sns.kdeplot(elem[:,0], elem[:,1], n_levels=2, cmaps=sequential_colors[i], shade=True, shade_lowest=False, alpha=.7, ax=ax, label=str(i))
        i += 1
    fig.legend()
    